In [1]:
"""
Created on Wed Aug  4 17:50:06 2021

@author: amol
"""

# set TF GPU memory growth so that it doesn't hog everything at once
import tensorflow as tf
physical_devices = tf.config.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(physical_devices[0], True)

from main import *
debug_mode = True  # faster experiments for debugging

# util for getting objects' fields' names
field_names = lambda x: list(vars(x).keys())

In [2]:
#Prepare the DataFrame that will be used downstream
dp = DataPreparer()
dp.createPCAs()
dp.sparsePCAs()
dp.zmixOrthogonalPCAs()
df = dp.getDataframe()

# TODO: add PCA from linear model first
df.to_csv('PCA_data.csv', index=False)

# currently passing dp eventually we want to abstract all the constants into 1 class
dm = DataManager(df, dp)

In [ ]:
'''
1. Run the GP Experiments
'''
import time

start=time.time()
exprExec = run_gp_experiments(dm, debug_mode=debug_mode)
print(f'duration: {time.time()-start}')

In [ ]:
'''
2. Run the Simple DNN Experiments
'''

exprExec = run_simple_dnn_experiments(dm, debug_mode=debug_mode)

In [3]:
'''
4. Run the PCDNN_v2 Experiments
'''

%debug exprExec = run_pcdnn_v2_experiments(dm, debug_mode=debug_mode)

NOTE: Enter 'c' at the ipdb>  prompt to continue execution.
> <string>(1)<module>()

ipdb> q


In [3]:
""" prepare PCDNNV2 for loading (from prior experiments) """

exprExec = PCDNNV2ExperimentExecutor()
exprExec.debug_mode = debug_mode

exprExec.setModelFactory(PCDNNV2ModelFactory())

Parent DNNModelFactory Instantiated


In [ ]:
""" prepare PCDNNV1 for loading (from prior experiments) """

exprExec = PCDNNV1ExperimentExecutor()
exprExec.debug_mode = debug_mode

exprExec.setModelFactory(PCDNNV1ModelFactory())

In [ ]:
""" prepare SimpleDNN for loading (from prior experiments) """

exprExec = DNNExperimentExecutor()
exprExec.debug_mode = debug_mode

exprExec.setModelFactory(SimpleDNNModelFactory())

In [4]:
dataType = 'randomequaltraintestsplit' #'frameworkincludedtrainexcludedtest'
inputType = 'AllSpecies'
dataSetMethod = f'{inputType}_{dataType}'
opscaler = 'PositiveLogNormal' #"MinMaxScaler"
ZmixPresent = 'Y'
concatenateZmix = 'Y'
kernel_constraint = 'N'
kernel_regularizer = 'N'
activity_regularizer = 'N'
noOfCpv = 4
noOfNeurons = 53

exprExec.modelFactory.loss='mse'
exprExec.modelFactory.activation_func='relu'
exprExec.modelFactory.dropout_rate=0.5
exprExec.debug_mode = False
exprExec.epochs_override = 10
exprExec.n_models_override = 1

# initialize experiment executor...
exprExec.dm = dm
exprExec.df_experimentTracker = pd.DataFrame()
exprExec.modelType = 'PCDNNV2'

exprExec.executeSingleExperiment(noOfNeurons,dataSetMethod,dataType,inputType,ZmixPresent,noOfCpv,concatenateZmix,kernel_constraint,
                                 kernel_regularizer,activity_regularizer,opscaler=opscaler)

--------------------self.build_and_compile_pcdnn_v2_model----------------------
53 4 N N N
Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
species_input (InputLayer)      [(None, 53)]         0                                            
__________________________________________________________________________________________________
zmix (InputLayer)               [(None, 1)]          0                                            
__________________________________________________________________________________________________
linear_embedding (Functional)   (None, 4)            216         species_input[0][0]              
__________________________________________________________________________________________________
concatenated_zmix_linear_embedd (None, 5)            0           zmix[0][0]                       
   

ipdb> n
> /home/dwyerdei/rom_project2/src/experiment_executor/error_manager.py(16)computeError()
     14         evaluation_df_1 = pd.DataFrame()
     15 
---> 16         evaluation_df_1['souener'] = Y_test.flatten()
     17 
     18         evaluation_df_1['souener_pred'] = Y_pred.flatten()

ipdb> n
> /home/dwyerdei/rom_project2/src/experiment_executor/error_manager.py(18)computeError()
     16         evaluation_df_1['souener'] = Y_test.flatten()
     17 
---> 18         evaluation_df_1['souener_pred'] = Y_pred.flatten()
     19 
     20         evaluation_df_1['souener_pred_L1'] = (evaluation_df_1['souener'] - evaluation_df_1['souener_pred']).abs()

ipdb> n
> /home/dwyerdei/rom_project2/src/experiment_executor/error_manager.py(20)computeError()
     18         evaluation_df_1['souener_pred'] = Y_pred.flatten()
     19 
---> 20         evaluation_df_1['souener_pred_L1'] = (evaluation_df_1['souener'] - evaluation_df_1['souener_pred']).abs()
     21 
     22         evaluation_df_1['so

BdbQuit: 

In [5]:
import numpy as np
print(np.log(2189882659.968396)/np.log(10))
print(np.log(2099652636.0322459)/np.log(10))
print(np.log(199414035386.85098)/np.log(10))
#vars(exprExec.dm.outputScaler.log_col_transformers[0])
#exprExec.df_err

print(np.exp(np.sqrt(0.0370)))


9.340420844751762
9.322147451527908
11.299755722062626
1.2120993310693264


In [ ]:
good_model = exprExec.model
print(good_model)

In [ ]:
import keras
help(keras.models.Model.fit)

In [ ]:
import model_analyzer.model_analysis as model_analysis
import importlib; importlib.reload(model_analysis)

bestModel, experimentSettings = exprExec.modelFactory.openBestModel()
print(f'\nexperimentSettings: {experimentSettings}')
print(f'\nbestModel.input_shape: {bestModel.input_shape}')
inspector = model_analysis.ModelInspector(exprExec.modelFactory, dm)

In [ ]:
n_repeats = 5 if debug_mode else 20
inspector.plot_permutation_feature_importance(n_repeats=n_repeats)

In [ ]:
inspector.plot_partial_dependence()

### Print & Record Linear Embeddings Output

In [ ]:
import numpy as np

def inspect_PCA(X):
    PCA_dict = {}
    PCA_dict['X'] = X
    PCA_dict['Var'] = PCA_dict['X'].var(axis=0)
    
    # sort by explained variance
    sort_idx = np.argsort(PCA_dict['Var'])[::-1]
    PCA_dict['X'] = PCA_dict['X'][:, sort_idx]
    PCA_dict['Var'] = PCA_dict['Var'][sort_idx]
    PCA_dict['CumVar'] = np.add.accumulate(PCA_dict['Var'])
    return PCA_dict


X, Y, rom, zmix = dm.getAllData()
FullDataset = inspect_PCA(X)
try:    
    linearAutoEncoder = exprExec.modelFactory.getLinearEncoder()
    X = linearAutoEncoder.predict(FullDataset['X'])
    DNN_PCA = inspect_PCA(X)
except KeyError:
    print('No Linear Auto Encoder!')

In [ ]:
#dm.createDataset
import matplotlib.pyplot as plt
import copy
dm_PurePCA = copy.deepcopy(dm)
dm_PurePCA.createTrainTestData(dataSetMethod='PurePCA_randomequalflamesplit',
                               numCpvComponents=2, ipscaler=None, opscaler=None)

X,Y,rom,zmix = dm_PurePCA.getAllData()
PurePCA = inspect_PCA(X)
print(PurePCA['Var'])
print(DNN_PCA['Var'])

plt.plot(PurePCA['CumVar']/FullDataset['Var'].sum(), color='r')
plt.plot(DNN_PCA['CumVar']/FullDataset['Var'].sum(), color='b')
plt.title('PCA Total Variance Comparison')